In [1]:
using StatisticalRethinking
using CmdStan, StanMCMCChain

ProjDir = rel_path("..", "scripts", "12")

d = CSV.read(rel_path( "..", "data",  "Kline.csv"), delim=';');
size(d) # Should be 10x5

(10, 5)

New col log_pop, set log() for population data

In [2]:
d[:log_pop] = map((x) -> log(x), d[:population]);
d[:society] = 1:10;

first(d, 5)

m12_6 = "
  data {
    int N;
    int T[N];
    int N_societies;
    int society[N];
    int P[N];
  }
  parameters {
    real alpha;
    vector[N_societies] a_society;
    real bp;
    real<lower=0> sigma_society;
  }
  model {
    vector[N] mu;
    target += normal_lpdf(alpha | 0, 10);
    target += normal_lpdf(bp | 0, 1);
    target += cauchy_lpdf(sigma_society | 0, 1);
    target += normal_lpdf(a_society | 0, sigma_society);
    for(i in 1:N) mu[i] = alpha + a_society[society[i]] + bp * log(P[i]);
    target += poisson_log_lpmf(T | mu);
  }
  generated quantities {
    vector[N] log_lik;
    {
    vector[N] mu;
    for(i in 1:N) {
      mu[i] = alpha + a_society[society[i]] + bp * log(P[i]);
      log_lik[i] = poisson_log_lpmf(T[i] | mu[i]);
    }
    }
  }
";

Define the Stanmodel and set the output format to :mcmcchain.

In [3]:
stanmodel = Stanmodel(name="m12.6",  model=m12_6, output_format=:mcmcchain);

Input data for cmdstan

In [4]:
m12_6_data = Dict("N" => size(d, 1), "T" => d[:total_tools], "N_societies" => 10, "society" => d[:society], "P" => d[:population]);

Sample using cmdstan

In [5]:
rc, chn, cnames = stan(stanmodel, m12_6_data, ProjDir, diagnostics=false, summary=false, CmdStanDir=CMDSTAN_HOME);

Describe the draws

In [6]:
describe(chn)

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
                   Mean           SD         Naive SE        MCSE         ESS    
         lp__ -39.317221750  3.0109981791 0.047608061384 0.0884960448 1000.000000
accept_stat__   0.948033762  0.0752345755 0.001189563086 0.0016050444 1000.000000
   stepsize__   0.048517150  0.0051318977 0.000081142428 0.0008216586   39.009752
  treedepth__   5.943750000  0.4523683198 0.007152571160 0.0183334062  608.832805
 n_leapfrog__  77.164000000 36.4131924465 0.575743125045 1.3282321391  751.570186
  divergent__   0.000000000  0.0000000000 0.000000000000 0.0000000000         NaN
     energy__  45.803708800  3.9372457248 0.062253320991 0.1101808790 1000.000000
        alpha   1.119863985  0.6918879371 0.010939708834 0.0189536333 1000.000000
  a_society.1  -0.203146213  0.2336603480 0.003694494492 0.0044914508 1000.000000
  a_society.2   0.036164170  0.2071934368 0.003276015882 0.00454

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*